In [110]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from IPython.display import Markdown
from scipy import stats
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from src.report import (
    t_test,
    f_test,
    general,
    general_assessment,
    other
)
from src.stats import LinearRegression, add_constant

In [111]:
%reload_ext autoreload
%autoreload 2

In [112]:
ALPHA = 0.05

# Множественная регрессия

In [113]:
DATA = """
No Qi(млн.шт.) Ii(тыс.руб). Mi(млн.руб) Pi(руб.)
1 199,94 55,81 6,16 41,55
2 200,56 56,34 7,19 42,8
3 199,02 53,86 7,6 42,08
4 201,87 56,55 9,7 40,51
5 199,18 54,95 7,98 45,33
6 200,31 56,52 7,75 42,74
7 202,68 56,96 10,31 41,49
8 201,86 58,12 8,14 42,47
9 200,71 55,98 9,27 42,4
10 200,07 55,87 9,72 41,48
11 202,77 58,59 9,12 40,46
12 199,16 56 6,78 41,01
13 201,37 57,11 8,12 41,87
14 200,37 55,52 8,1 41,01
15 200,32 55,88 9,46 43,53
16 199,51 55,04 7,43 43,26
17 200,01 56 8,16 42,73
18 200,09 55,43 8,7 44,04
19 201,79 56,48 8,75 41,31
20 201,46 56,93 6,98 41,43
21 200,13 56,97 7,21 42,55
22 201,06 56,13 9,38 43,03
23 201,24 57,55 8,49 40,86
24 201,93 57,48 8,34 40,83
25 200,94 56,61 8,62 42,09
26 200,07 55,18 8,69 42,46
27 200,07 56,22 8 41,9
28 201,74 56,96 7,97 40,18
29 201,27 56,58 9,3 41,36
30 201,65 57,47 9,09 42,71
"""

DATA = [map(lambda x: float(x.replace(',', '.')), row.split()[1:]) for row in DATA.strip().split('\n')[1:]]

In [114]:
DATA = pd.DataFrame(DATA, columns=['Q', 'I', 'M', 'P'])
DATA

,Q,I,M,P
0,199.94,55.81,6.16,41.55
1,200.56,56.34,7.19,42.80
2,199.02,53.86,7.60,42.08
3,201.87,56.55,9.70,40.51
4,199.18,54.95,7.98,45.33
5,200.31,56.52,7.75,42.74
6,202.68,56.96,10.31,41.49
7,201.86,58.12,8.14,42.47
8,200.71,55.98,9.27,42.40
9,200.07,55.87,9.72,41.48


In [115]:

X = add_constant(DATA[['I', 'M', 'P']])
y = DATA['Q']

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.98, shuffle=False)
X_train

,const,I,M,P
0,1.0,55.81,6.16,41.55
1,1.0,56.34,7.19,42.80
2,1.0,53.86,7.60,42.08
3,1.0,56.55,9.70,40.51
4,1.0,54.95,7.98,45.33
5,1.0,56.52,7.75,42.74
6,1.0,56.96,10.31,41.49
7,1.0,58.12,8.14,42.47
8,1.0,55.98,9.27,42.40
9,1.0,55.87,9.72,41.48


In [117]:
model = LinearRegression(y_train, X_train)

In [118]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Q   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     38.86
Date:                Sat, 18 Dec 2021   Prob (F-statistic):           1.45e-09
Time:                        11:32:21   Log-Likelihood:                -15.892
No. Observations:                  29   AIC:                             39.78
Df Residuals:                      25   BIC:                             45.25
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        166.6738      8.023     20.774      0.0

In [119]:
general(model)

$ y_i = a + b_{1} x_{i1} + b_{2} x_{i2} + b_{3} x_{i3} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} + \hat{b_{2}} x_{i2} + \hat{b_{3}} x_{i3} $<br>
<br>
$ \hat{y_i} = 166.674 + 0.679 x_{i1} + 0.376 x_{i2} + -0.174 x_{i3} $<br>
<br>
$ S_\hat{a} = 8.023 $<br>
$ S_{\hat{b_{1}}} = 0.101 $<br>
$ S_{\hat{b_{2}}} = 0.088 $<br>
$ S_{\hat{b_{3}}} = 0.085 $

In [120]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 0.451 $<br>
$ 30 \% < 45.08_{S_{ei}} \% \rightarrow $ оценка: <b>0/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.823 $<br>
$ 0.8 < 0.823 < 0.9 \rightarrow $ оценка: <b>8/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 0.163 $<br>
$ 0 \% < 0.163 < 2 \% \rightarrow $ оценка: <b>10/10</b>.

In [121]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 38.856 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 1.926 $
<br><br>
$ 1.45e-09 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

In [122]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.060 $
<br>

<br>
$ t_{a} = 20.774 $, $ p\text{-}value_{t_{a}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = 6.722 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{2}} = 4.254 $, $ p\text{-}value_{t_{b_{2}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{3}} = -2.043 $, $ p\text{-}value_{t_{b_{3}}} = 0.052 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.

In [123]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \beta_j = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \beta_{1} = 0.659 $<br>
$ \beta_{2} = 0.363 $<br>
$ \beta_{3} = -0.198 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \Delta_{1} = 0.653 $<br>
$ \Delta_{2} = 0.214 $<br>
$ \Delta_{3} = 0.132 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ Э_{1} = 0.190 $<br>
$ Э_{2} = 0.016 $<br>
$ Э_{3} = -0.036 $

In [124]:
# С ростом дохода потребителей на 1% объем продаж увеличится на 0.19%.
# С ростом затрат на рекламу на 1% объем продаж увеличится на 0.016%.
# С ростом цены товара потребителей на 1% объем продаж уменьшится на 0.036%.

### Интерпретация параметров

In [125]:
# С ростом дохода потребителей на 1000 рублей, объем продаж вырастет на 678 единиц.
# Объем продаж больше всего зависит от дохода потребителей, при этом доход потребителей растет медленнее объема продаж.
# Чем больше затрат на рекламу, тем больше объем продаж (на 1 млн - 375 тыс.).
# Чем выше цена товара, тем меньше желание покупателей приобретать товар.

## Мультиколлинеарность

In [126]:
DATA.corr()

,Q,I,M,P
Q,1.000000,0.822532,0.498325,-0.522217
I,0.822532,1.000000,0.191158,-0.459786
M,0.498325,0.191158,1.000000,-0.054637
P,-0.522217,-0.459786,-0.054637,1.000000


### Метод вспомогательных регрессий

In [127]:
I__M_P_results = sm.OLS(X['I'], X[['M', 'P', 'const']]).fit()
M__I_P_results = sm.OLS(X['M'], X[['I', 'P', 'const']]).fit()
P__M_I_results = sm.OLS(X['P'], X[['M', 'I', 'const']]).fit()
pd.DataFrame(
    [
        [I__M_P_results.fvalue, M__I_P_results.fvalue, P__M_I_results.fvalue],
        [I__M_P_results.f_pvalue, M__I_P_results.f_pvalue, P__M_I_results.f_pvalue],
    ],
    index=['F', 'p-value'],
    columns=X.columns[1:]
)
# фактор M не приводит к мультиколлинеарности

,I,M,P
F,4.241078,0.532445,3.643971
p-value,0.025019,0.593206,0.039719


### VIF

In [128]:
pd.DataFrame(
    [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])],
    index=X.columns[1:],
    columns=['vif']
)
# ни один из факторов не вызывает мультиколлинеарности

,vif
I,1.314154
M,1.039440
P,1.269924


### Тест Фаррара-Глоубера.

In [129]:
R = X[X.columns[1:]].corr()
n = len(X)
k = len(R)

In [130]:
fg_value = -(n - 1 - (1 / 6) * (2 * k + 5)) * np.log(np.linalg.det(R))
print(fg_value)

chi2_critical = stats.chi2.ppf(1 - ALPHA, k * (k - 1) / 2)
print(chi2_critical)

7.502965925902487
7.814727903251179


In [131]:
Markdown(rf"""
$ FG = {fg_value:.2f} < X^2_{{крит}} = {chi2_critical:.2f} \rightarrow $ гипотеза H0 **принимается** -
в массиве объясняющих переменных отсутствует мультиколлинеарность.
""")


$ FG = 7.50 < X^2_{крит} = 7.81 \rightarrow $ гипотеза H0 **принимается** -
в массиве объясняющих переменных отсутствует мультиколлинеарность.


In [132]:
f_value = (n - k - 1) * (np.diagonal(np.linalg.inv(R)) - 1) / k
f_value = pd.Series(f_value, index=R.columns, name='F')
f_value

I    2.722668
M    0.341816
P    2.339340
Name: F, dtype: float64

In [133]:
f_critical = stats.f.ppf(1 - ALPHA, k, n - k - 1)
f_critical

2.9751539639733933

In [134]:
print(f_value > f_critical)
Markdown("""
**False** - исследуемая экзогенная переменная **не мультиколлинеарна** с другими
""")

I    False
M    False
P    False
Name: F, dtype: bool



**False** - исследуемая экзогенная переменная **не мультиколлинеарна** с другими


In [135]:
pairs = [(0, 1), (0, 2), (1, 2), (2, 1), (2, 0), (1, 0)]
private_corr = pd.Series(dtype=object)
for i, j in pairs:
    pair = f'{i=},{j=}'
    private_corr[pair] = -np.linalg.inv(R)[i, j] / np.sqrt(np.linalg.inv(R)[i, i] * np.linalg.inv(R)[j, j])

private_corr

i=0,j=1    0.187252
i=0,j=2   -0.458469
i=1,j=2    0.038151
i=2,j=1    0.038151
i=2,j=0   -0.458469
i=1,j=0    0.187252
dtype: float64

In [136]:
t_value = private_corr * np.sqrt(n - k - 1) / np.sqrt(1 - private_corr * private_corr)
t_value

i=0,j=1    0.971994
i=0,j=2   -2.630485
i=1,j=2    0.194675
i=2,j=1    0.194675
i=2,j=0   -2.630485
i=1,j=0    0.971994
dtype: float64

In [137]:
t_critical = stats.t.ppf(1 - ALPHA, n - k - 1)
t_critical

1.7056179197592727

In [138]:
summary = pd.DataFrame((abs(t_value) > t_critical).rename('bool'))  # noqa
summary['desc'] = summary['bool'].map(lambda x: 'коррел. ' + ('значима' if x else 'незначима'))
summary

,bool,desc
"i=0,j=1",False,коррел. незначима
"i=0,j=2",True,коррел. значима
"i=1,j=2",False,коррел. незначима
"i=2,j=1",False,коррел. незначима
"i=2,j=0",True,коррел. значима
"i=1,j=0",False,коррел. незначима


### AIC, t-статистики

In [139]:
Q__I_M_results = sm.OLS(y, X[['I', 'M', 'const']]).fit()
print(Q__I_M_results.pvalues)
print(Q__I_M_results.aic)

I        3.637534e-09
M        4.301630e-04
const    1.338504e-22
dtype: float64
42.818132287457175


In [140]:
Q__M_P_results = sm.OLS(y, X[['M', 'P', 'const']]).fit()
print(Q__M_P_results.pvalues)
print(Q__M_P_results.aic)

M        1.919395e-03
P        1.193681e-03
const    9.655726e-26
dtype: float64
70.26011068715788


In [141]:
Q__I_P_results = sm.OLS(y, X[['I', 'P', 'const']]).fit()
print(Q__I_P_results.pvalues)
print(Q__I_P_results.aic)

I        1.092190e-06
P        1.337349e-01
const    5.138327e-16
dtype: float64
54.29422934643955


In [142]:
Q__I_results = sm.OLS(y, X[['I', 'const']]).fit()
print(Q__I_results.pvalues)
print(Q__I_results.aic)

I        2.453466e-08
const    1.372071e-20
dtype: float64
54.83902805574014


In [143]:
Q__P_results = sm.OLS(y, X[['P', 'const']]).fit()
print(Q__P_results.pvalues)
print(Q__P_results.aic)

P        3.074877e-03
const    3.122417e-25
dtype: float64
79.14827747712164


In [144]:
Q__M_results = sm.OLS(y, X[['M', 'const']]).fit()
print(Q__M_results.pvalues)
print(Q__M_results.aic)

M        5.068864e-03
const    3.773283e-41
dtype: float64
80.13753121167184


In [145]:
# по AIC лучше использовать модель: Q = I + M (42.8)

### Прогнозирование

In [146]:
y_pred_test_intervals = model.results.get_prediction(X_test).summary_frame(ALPHA)
y_pred_test_intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
29,201.682771,0.179731,201.312607,202.052935,200.683253,202.68229


In [147]:
print('Для среднего')
all((y_pred_test_intervals['mean_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['mean_ci_upper']))  # noqa

Для среднего


True

In [148]:
print('Для индивидуального')
all((y_pred_test_intervals['obs_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['obs_ci_upper']))  # noqa

Для индивидуального


True

In [149]:
# модель адекватна

# Нелинейная модель

In [150]:
DATA_ = """
Фирма Qi Li Ki
1 58 100 118
2 59 104 140
3 55 90 85
4 74 102 140
5 80 115 179
6 60 96 96
7 69 98 130
8 72 99 140
9 73 101 145
10 67 98 138
11 63 96 106
12 58 97 90
13 71 98 124
14 70 100 129
"""

DATA_ = [map(int, row.split()[1:]) for row in DATA_.strip().split('\n')[1:]]

In [151]:
DATA_ = pd.DataFrame(DATA_, columns=['Q', 'L', 'K'])
DATA_

,Q,L,K
0,58,100,118
1,59,104,140
2,55,90,85
3,74,102,140
4,80,115,179
5,60,96,96
6,69,98,130
7,72,99,140
8,73,101,145
9,67,98,138


In [152]:
X = add_constant(np.log10(DATA_[['L', 'K']]))
y = np.log10(DATA_['Q'])

In [153]:
model = LinearRegression(y, X)

In [154]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Q   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     11.72
Date:                Sat, 18 Dec 2021   Prob (F-statistic):            0.00188
Time:                        11:32:37   Log-Likelihood:                 30.706
No. Observations:                  14   AIC:                            -55.41
Df Residuals:                      11   BIC:                            -53.50
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4154      1.040      1.362      0.2

D:\IT\Coding\GitHub Projects\My-University\III Курс\(python) Эконометрика\venv\Lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [155]:
general(model)

$ y_i = a + b_{1} x_{i1} + b_{2} x_{i2} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} + \hat{b_{2}} x_{i2} $<br>
<br>
$ \hat{y_i} = 1.415 + -0.352 x_{i1} + 0.529 x_{i2} $<br>
<br>
$ S_\hat{a} = 1.040 $<br>
$ S_{\hat{b_{1}}} = 0.663 $<br>
$ S_{\hat{b_{2}}} = 0.173 $

In [156]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 0.030 $<br>
$ 2 \% < 3.04_{S_{ei}} \% < 5 \% \rightarrow $ оценка: <b>8/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.681 $<br>
$ 0.5 < 0.681 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 1.150 $<br>
$ 0 \% < 1.150 < 2 \% \rightarrow $ оценка: <b>10/10</b>.

In [157]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 11.717 $<br>
$ p\text{-}value_F = 0.002 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.739 $
<br><br>
$ 0.00188 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

In [158]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.201 $
<br>

<br>
$ t_{a} = 1.362 $, $ p\text{-}value_{t_{a}} = 0.201 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{1}} = -0.531 $, $ p\text{-}value_{t_{b_{1}}} = 0.606 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{2}} = 3.064 $, $ p\text{-}value_{t_{b_{2}}} = 0.011 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

In [159]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \beta_j = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \beta_{1} = -0.166 $<br>
$ \beta_{2} = 0.960 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \Delta_{1} = 1.156 $<br>
$ \Delta_{2} = -0.156 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ Э_{1} = -0.386 $<br>
$ Э_{2} = 0.608 $

### Обратная замена

In [160]:
# Q = 10^1.4 * L^(-0.35) * K^0.53
#     (1.04)    (0.66)     (0.17)

### Интерпретация параметров

In [161]:
# С увеличением трудозатрат L, выпуск Q уменьшается.
# С увеличением капиталовложения K, выпуск Q увеличивается.

# Письменные задачи

1\. Рассматривается регрессионная модель, построенная на основе 100 наблюдений.

Используя тест Фаррара-Глоубера определите есть ли в массиве объясняющих переменных мультиколлинеарность.

Рассчитайте F-критерии теста Фаррара-Глоубера и определите, какая переменная больше влияет на общую мультиколлинеарность.

Рассчитайте частные коэффициенты корреляции.

In [162]:
R_inv = np.array([
    [2.9, -1.043, -1.633],
    [-2.232, 2.683, -0.131],
    [-0.142, -1.415, 1.96]
])

In [163]:
R = np.linalg.inv(R_inv)
n = 100
k = 3

In [164]:
fg_value = -(n - 1 - (1 / 6) * (2 * k + 5)) * np.log(np.linalg.det(R))
print(fg_value)

chi2_critical = stats.chi2.ppf(1 - ALPHA, k * (k - 1) / 2)
print(chi2_critical)

142.86900020884826
7.814727903251179


In [165]:
Markdown(rf"""
$ FG = {fg_value:.2f} > X^2_{{крит}} = {chi2_critical:.2f} \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.
""")


$ FG = 142.87 > X^2_{крит} = 7.81 \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.


In [166]:
f_value = (n - k - 1) * (np.diagonal(R_inv) - 1) / k
f_value = pd.Series(f_value, index=[f'x{i}' for i in range(len(f_value))], name='F')
f_value

x0    60.800
x1    53.856
x2    30.720
Name: F, dtype: float64

In [167]:
f_critical = stats.f.ppf(1 - ALPHA, k, n - k - 1)
f_critical

2.6993925975521815

In [168]:
print(f_value > f_critical)
Markdown("""
True - исследуемая экзогенная переменная мультиколлинеарна с другими
""")

x0    True
x1    True
x2    True
Name: F, dtype: bool



True - исследуемая экзогенная переменная мультиколлинеарна с другими


In [169]:
pairs = [(0, 1), (0, 2), (1, 2), (2, 1), (2, 0), (1, 0)]
private_corr = pd.Series(dtype=object)
for i, j in pairs:
    pair = f'{i=},{j=}'
    private_corr[pair] = -R_inv[i, j] / np.sqrt(R_inv[i, i] * R_inv[j, j])

private_corr

i=0,j=1    0.373917
i=0,j=2    0.684950
i=1,j=2    0.057126
i=2,j=1    0.617047
i=2,j=0    0.059561
i=1,j=0    0.800175
dtype: float64

In [170]:
t_value = private_corr * np.sqrt(n - k - 1) / np.sqrt(1 - private_corr * private_corr)
t_value

i=0,j=1     3.950157
i=0,j=2     9.211106
i=1,j=2     0.560633
i=2,j=1     7.682800
i=2,j=0     0.584613
i=1,j=0    13.071878
dtype: float64

In [171]:
t_critical = stats.t.ppf(1 - ALPHA, n - k - 1)
t_critical

1.6608814403008005

In [172]:
summary = pd.DataFrame((abs(t_value) > t_critical).rename('bool'))  # noqa
summary['desc'] = summary['bool'].map(lambda x: 'коррел. ' + ('значима' if x else 'незначима'))
summary

,bool,desc
"i=0,j=1",True,коррел. значима
"i=0,j=2",True,коррел. значима
"i=1,j=2",False,коррел. незначима
"i=2,j=1",True,коррел. значима
"i=2,j=0",False,коррел. незначима
"i=1,j=0",True,коррел. значима


2\. В результате оценивания регрессионной модели по выборочным данным объемом 200 наблюдения получен результат.

В модели исследователь подозревает наличие мультиколлинеарности. С помощью коэффициентов вздутия проверьте данное предположение.

In [173]:
rsquared_x1__x2_x3 = 0.6
rsquared_x2__x1_x3 = 0.55
rsquared_x3__x1_x2 = 0.66

vif_list = [1 / (1 - rsquared_x1__x2_x3), 1 / (1 - rsquared_x2__x1_x3), 1 / (1 - rsquared_x3__x1_x2)]
vif = pd.DataFrame(vif_list, [f'vif_{i}' for i in range(len(vif_list))], columns=['vif'])
vif[' > 2'] = vif['vif'] > 2
vif[' < 10'] = vif['vif'] < 10
vif['desc'] = vif[' < 10'].map(lambda x: ('не ' if x else '') + 'приводит к мультиколлинеарности')
vif

,vif,> 2,< 10,desc
vif_0,2.500000,True,True,не приводит к мультиколлинеарности
vif_1,2.222222,True,True,не приводит к мультиколлинеарности
vif_2,2.941176,True,True,не приводит к мультиколлинеарности


3\. В результате оценивания регрессионной модели по выборочным данным объемом 200 наблюдения получен результат.

Вам известен определитель матрицы межфакторных корреляций (xij друг с другом) $ det(R) = 0,562 $. Используя тест Фаррара-Глоубера определите есть ли в массиве объясняющих переменных мультиколлинеарность.

In [174]:
det_R = 0.562
n = 200
k = 3

In [175]:
fg_value = -(n - 1 - (2 * k + 5) / 6) * np.log(det_R)
print(fg_value)

chi2_critical = stats.chi2.ppf(1 - ALPHA, k * (k - 1) / 2)
print(chi2_critical)

113.61796776860524
7.814727903251179


In [176]:
Markdown(rf"""
$ FG = {fg_value:.2f} > X^2_{{крит}} = {chi2_critical:.2f} \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.
""")


$ FG = 113.62 > X^2_{крит} = 7.81 \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.
